In [2]:
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np
import copy

# Custom algorithm steps

## RHF conditions

In [2]:
def updateFockMatrixToRHFStructure(environment):
    """
    A function used to update the generalized fock matrix to the RHF structure:
    
    (F_R, 0)
    (0, F_R)
    """
    
    # Extract the latest Fock matrix out of the solver's environment.
    fock_matrix_generalized = environment.fock_matrices[-1].parameters()
    fock_matrix = copy.deepcopy(fock_matrix_generalized)
    
    
    # In order to select the wanted blocks from the Fock matrix, we need the dimensions.
    dim = int(np.shape(fock_matrix)[0]/2)
    
    # Create zero blocks and fill them in.
    zero_block = np.zeros((dim, dim))
    
    fock_matrix[0:dim, dim:] = zero_block
    fock_matrix[dim:, 0:dim] = zero_block
    
    # Select the restricted part of the fock matrix and make sure the two F_R blocks are identical.
    fock_restricted = fock_matrix[0:dim, 0:dim]
    
    fock_matrix[dim:, dim:] = fock_restricted

    fock_operator = gqcpy.ScalarGSQOneElectronOperator_d(fock_matrix)
    
    # Replace the current fock matrix in the environment
    environment.replace_current_fock_matrix(fock_operator)

In [3]:
modification_RHF_F = gqcpy.FunctionalStep_GHFSCFEnvironment_d(updateFockMatrixToRHFStructure, description="Modify the Fock matrix to reflect a generalized representation of an RHF Fock matrix.")

In [4]:
def updateCoefficientsToRHFStructure(environment):
    """
    A function used to update the generalized coefficient matrix to the RHF structure:
    
    (C_R, 0)
    (0, C_R)
    
    and uses this to transform the MO density matrix to AO basis.
    """
        
    # Extract the latest coefficient matrix out of the solver's environment.
    coefficient_matrix_generalized = environment.coefficient_matrices[-1].matrix()
    coefficient_matrix = copy.deepcopy(coefficient_matrix_generalized)
    
    
    # In order to select the wanted blocks from the coefficient matrix, we need the dimensions.
    dim = int(np.shape(coefficient_matrix)[0]/2)
    
    # Create zero blocks and fill them in.
    zero_block = np.zeros((dim, dim))
    
    coefficient_matrix[0:dim, dim:] = zero_block
    coefficient_matrix[dim:, 0:dim] = zero_block
    
    # Select the restricted part of the coeffcient matrix and make sure the two C_R blocks are identical.
    coefficient_restricted = coefficient_matrix[0:dim, 0:dim]
    
    coefficient_matrix[dim:, dim:] = coefficient_restricted

    coefficients = gqcpy.GTransformation_d(coefficient_matrix)
        
    # Replace the current coefficient matrix in the environment
    environment.replace_current_coefficient_matrix(coefficients)

In [5]:
modification_RHF_C = gqcpy.FunctionalStep_GHFSCFEnvironment_d(updateCoefficientsToRHFStructure, description="Modify the Coefficient matrix to reflect a generalized representation of an RHF Coefficient matrix.")

In [6]:
def updateDensityToRHFStructure(environment):
    """
    A function used to update the generalized coefficient matrix to the RHF structure:
    
    (C_R, 0)
    (0, C_R)
    
    and uses this to transform the MO density matrix to AO basis.
    """
    
    # Extract the latest coefficient matrix out of the solver's environment.
    coefficient_matrix_generalized = environment.coefficient_matrices[-1].matrix()
    
    coefficient_matrix = copy.deepcopy(coefficient_matrix_generalized)
    
    # We need the dimensions.
    dim = int(np.shape(coefficient_matrix)[0]/2)
    
    # create the RHF MO density matrix and transform it with the new coefficient matrix
    density_MO = np.zeros((dim * 2, dim * 2))
    
    if environment.N % 2 != 0:
        raise Exception("RHF modification cannot be used for an odd number of electrons.")
      
    electron_pairs = int(environment.N / 2)
    diagonal_occupied = np.ones((electron_pairs, electron_pairs)) * 2
    
    density_MO[0:electron_pairs, 0:electron_pairs] = diagonal_occupied
    
    
    transformation = gqcpy.GTransformation_d(coefficient_matrix)
    density = gqcpy.G1DM_d(density_MO).transformed(transformation)
    
    # Replace the current density matrix in the environment
    environment.replace_current_density_matrix(density)

In [7]:
modification_RHF_D = gqcpy.FunctionalStep_GHFSCFEnvironment_d(updateDensityToRHFStructure, description="Replace the density matrix with a new one calculated from the orthonormal 1DM representation, transformed to AO basis with the modified coefficient matrix.")

## UHF conditions

In [8]:
def updateFockMatrixToUHFStructure(environment):
    """
    A function used to update the generalized fock matrix to the UHF structure:
    
    (F_a, 0)
    (0, F_b)
    """
    
    # Extract the latest Fock matrix out of the solver's environment.
    fock_matrix_generalized = environment.fock_matrices[-1].parameters()
    fock_matrix = copy.deepcopy(fock_matrix_generalized)
    
    
    # In order to select the wanted blocks from the Fock matrix, we need the dimensions.
    dim = int(np.shape(fock_matrix)[0]/2)
    
    # Create zero blocks and fill them in.
    zero_block = np.zeros((dim, dim))
    
    fock_matrix[0:dim, dim:] = zero_block
    fock_matrix[dim:, 0:dim] = zero_block

    fock_operator = gqcpy.ScalarGSQOneElectronOperator_d(fock_matrix)
        
    # Replace the current fock matrix in the environment
    environment.replace_current_fock_matrix(fock_operator)

In [9]:
modification_UHF_F = gqcpy.FunctionalStep_GHFSCFEnvironment_d(updateFockMatrixToUHFStructure, description="Modify the Fock matrix to reflect a generalized representation of an UHF Fock matrix.")

In [10]:
def updateCoefficientsToUHFStructure(environment):
    """
    A function used to update the generalized coefficient matrix to the UHF structure:
    
    (C_a, 0)
    (0, C_b)
    
    and uses this to transform the MO density matrix to AO basis.
    """
    
    # Extract the latest coefficient matrix out of the solver's environment.
    coefficient_matrix_generalized = environment.coefficient_matrices[-1].matrix()
    coefficient_matrix = copy.deepcopy(coefficient_matrix_generalized)

    # In order to select the wanted blocks from the coefficient matrix, we need the dimensions.
    dim = int(np.shape(coefficient_matrix)[0]/2)
    
    # Create zero blocks and fill them in.
    zero_block = np.zeros((dim, dim))
    
    coefficient_matrix[0:dim, dim:] = zero_block
    coefficient_matrix[dim:, 0:dim] = zero_block
    
    coefficients = gqcpy.GTransformation_d(coefficient_matrix)
    
    # Replace the current coefficient matrix in the environment
    environment.replace_current_coefficient_matrix(coefficients)

In [127]:
modification_UHF_C = gqcpy.FunctionalStep_GHFSCFEnvironment_d(updateCoefficientsToUHFStructure, description="Modify the Coefficient matrix to reflect a generalized representation of an UHF Coefficient matrix.")

In [3]:
x = np.ones((3,3))
print(x)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [ ]:
val, vec = np.linalg.eig(x)

In [128]:
def updateDensityToUHFStructure(environment):
    """
    A function used to update the generalized coefficient matrix to the UHF structure:
    
    (C_a, 0)
    (0, C_b)
    
    and uses this to transform the MO density matrix to AO basis.
    """
    if len(environment.coefficient_matrices) < 2:
        print("Cannot yet update density")
        return
    
    # Extract the latest Coefficient matrix out of the solver's environment.
    generalized_coefficient_matrix = environment.coefficient_matrices[-1].matrix()  
    coefficient_matrix = copy.deepcopy(generalized_coefficient_matrix)
    
    # We need the dimensions.
    dim = int(np.shape(coefficient_matrix)[0]/2)
    
    # create the RHF MO density matrix and transform it with the new coefficient matrix
    density_MO = np.zeros((dim * 2, dim * 2))
    
    if environment.N % 2 == 0:
        Na = int(environment.N / 2)
        Nb = int(environment.N / 2)
    else:
        Na = int(np.ceil(environment.N / 2))
        Nb = int(np.floor(environment.N / 2))
    
    diagonal_occupied_a = np.ones((Na, Na))
    diagonal_occupied_b = np.ones((Nb, Nb))
    
    density_MO[0:Na, 0:Na] = diagonal_occupied_a
    density_MO[dim:Nb, dim:Nb] = diagonal_occupied_b
    
    transformation = gqcpy.GTransformation_d(coefficient_matrix)
    
    density = gqcpy.G1DM_d(density_MO).transformed(transformation)
    
    # Replace the current density matrix in the environment
    environment.replace_current_density_matrix(density)

In [129]:
modification_UHF_D = gqcpy.FunctionalStep_GHFSCFEnvironment_d(updateDensityToUHFStructure, description="Replace the density matrix with a new one calculated from the orthonormal 1DM representation, transformed to AO basis with the modified coefficient matrix.")

# The GHF algorithm

In [130]:
def GHF_calculation(molecule, basis_set, modification_type=None,):
    """
    A GHF algorithm that can be modified to a generalized RHF/UHF algorithm. 
    
    :param molecule:          The molecule for the calculation.
    :param basis_set:         The basis set in which the calculation is performed.
    :param modification type: The type of adjustment inserted into the algorithm. Default is None, but can be either RHF or UHF.
    
    :return: A gqcpy.QCStructure of the performed calculation, the ground state parameters and the ground state energy.
    """
    # Raise an exception when a wrong modification type is used.
    if modification_type != 'UHF' and modification_type != 'RHF' and modification_type != None:
        raise Exception('Invalid argument for modification_type. Use either RHF, UHF or the default None.')
        
        
    # Set up the initial required variables for a calculation.
    N = molecule.numberOfElectrons()
    basis = gqcpy.GSpinorBasis_d(molecule, basis_set)
    
    S = basis.quantize(gqcpy.OverlapOperator())
    
    fq_hamiltonian = gqcpy.FQMolecularHamiltonian(molecule)
    gsq_hamiltonian = basis.quantize(fq_hamiltonian) 
    
    # Create the solver, potentially with modifications. Print the steps in the algorithm to show what you are doing.
    environment = gqcpy.GHFSCFEnvironment_d.WithCoreGuess(N, gsq_hamiltonian, S)
    solver = gqcpy.GHFSCFSolver_d.Plain(1.0e-04, 1000)
    
    if modification_type == 'RHF':
        solver.insert(modification_RHF_C, 3)
        solver.insert(modification_RHF_D, 1)
        print(solver.description())
        
    elif modification_type == 'UHF':
        #solver.insert(modification_UHF_F, 2)
        solver.insert(modification_UHF_C, 3)
        print(solver.description())
        
    else:
        print(solver.description())
    
    # Calculate the QC structure.
    qc_structure = gqcpy.GHF_d.optimize(solver, environment)
    
    # Calculate and print the ground state energy.
    nuc_rep = gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
    ground_state_energy = qc_structure.groundStateEnergy() + nuc_rep
    
    print("                                                                                      ")
    print("                                                                                      ")
    print("======================================================================================")
    print("The ground state energy of the calculation: " + str(ground_state_energy) + " (hartree)")
    print("======================================================================================")

    
    # Calculate the ground state parameters.
    ground_state_parameters = qc_structure.groundStateParameters()
    
    return qc_structure, ground_state_energy, ground_state_parameters

# A test calculation

## H3 ring vs UHF modified H3 ring

In [131]:
# A H3 ring, 1 angstrom apart.
H3 = gqcpy.Molecule.HRingFromDistance(3, 1.889, 0)

In [132]:
structH3, energy_H3, paramH3 = GHF_calculation(H3, "STO-3G")

An iterative algorithm (with a maximum of 1000 iterations) consisting of the following steps:
An algorithmic step consisting of 4 algorithmic steps:
	1. Calculate the current GHF density matrix and place it in the environment.
	2. Calculate the current GHF Fock matrix (expressed in the scalar/AO basis) and place it in the environment.
	3. Solve the generalized eigenvalue problem for the most recent scalar/AO Fock matrix. Add the associated coefficient matrix and orbital energies to the environment.
	4. Calculate the current electronic GHF energy and place it in the environment.

With the following convergence criterion:
A convergence criterion that checks if the norm of the difference of two iterates (the GHF density matrix in AO basis) is converged, with a tolerance of 1.00e-04.
                                                                                      
                                                                                      
The ground state energy of the calc

In [133]:
structH3mod, energy_H3mod, paramH3mod = GHF_calculation(H3, "STO-3G", modification_type='UHF')

An iterative algorithm (with a maximum of 1000 iterations) consisting of the following steps:
An algorithmic step consisting of 5 algorithmic steps:
	1. Calculate the current GHF density matrix and place it in the environment.
	2. Calculate the current GHF Fock matrix (expressed in the scalar/AO basis) and place it in the environment.
	3. Solve the generalized eigenvalue problem for the most recent scalar/AO Fock matrix. Add the associated coefficient matrix and orbital energies to the environment.
	4. Modify the Coefficient matrix to reflect a generalized representation of an UHF Coefficient matrix.
	5. Calculate the current electronic GHF energy and place it in the environment.

With the following convergence criterion:
A convergence criterion that checks if the norm of the difference of two iterates (the GHF density matrix in AO basis) is converged, with a tolerance of 1.00e-04.


ValueError: cannot reshape array of size 1 into shape (3,3)

### Compare coefficient matrices

In [68]:
print(paramH3.expansion().matrix())
print("                                                                                      ")
print("======================================================================================")
print("                                                                                      ")
print(paramH3mod.expansion().matrix())

[[-4.59977538e-01  0.00000000e+00 -9.96838572e-01 -5.35322745e-01
   0.00000000e+00  0.00000000e+00]
 [-4.59929659e-01  0.00000000e+00  9.96615883e-01 -5.35778319e-01
   0.00000000e+00  0.00000000e+00]
 [-3.01690986e-01  0.00000000e+00  2.77730930e-04  1.18356216e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.80671980e-01  0.00000000e+00  0.00000000e+00
  -6.47473718e-01  9.96914209e-01]
 [ 0.00000000e+00  2.80770111e-01  0.00000000e+00  0.00000000e+00
  -6.48006667e-01 -9.96540225e-01]
 [ 0.00000000e+00  6.38663729e-01 -0.00000000e+00  0.00000000e+00
   1.04112703e+00 -2.46905416e-04]]
                                                                                      
                                                                                      


NameError: name 'paramH3mod' is not defined

### Compare orbitale energies

In [75]:
print(paramH3.orbitalEnergies())
print("                                                                                      ")
print("======================================================================================")
print("                                                                                      ")
print(paramH3mod.orbitalEnergies())

[-0.70607023 -0.58135742 -0.11123423  0.40413598  0.44076304  0.60050836]
                                                                                      
                                                                                      
[-0.70607023 -0.58135742 -0.11123423  0.40413598  0.44076304  0.60050836]


In [76]:
print(paramH3.occupiedOrbitalEnergies())

[-0.7060702326886801, -0.5813574180920079, -0.11123423288355501]


## H4 ring vs RHF modified H4 ring

In [77]:
# A H4 ring, 1 angstrom apart.
H4 = gqcpy.Molecule.HRingFromDistance(4, 1.889, 0)

In [78]:
structH4, energy_H4, paramH4 = GHF_calculation(H4, "sto-3g")

An iterative algorithm (with a maximum of 1000 iterations) consisting of the following steps:
An algorithmic step consisting of 4 algorithmic steps:
	1. Calculate the current GHF density matrix and place it in the environment.
	2. Calculate the current GHF Fock matrix (expressed in the scalar/AO basis) and place it in the environment.
	3. Solve the generalized eigenvalue problem for the most recent scalar/AO Fock matrix. Add the associated coefficient matrix and orbital energies to the environment.
	4. Calculate the current electronic GHF energy and place it in the environment.

With the following convergence criterion:
A convergence criterion that checks if the norm of the difference of two iterates (the GHF density matrix in AO basis) is converged, with a tolerance of 1.00e-04.
                                                                                      
                                                                                      
The ground state energy of the calc

In [ ]:
structH4mod, energy_H4mod, paramH4mod = GHF_calculation(H4, "sto-3g", "RHF")

In [ ]:
print(paramH4.expansion().matrix())

### Check orbital energies

In [ ]:
print(paramH4.orbitalEnergies())
print("                                                                                      ")
print("======================================================================================")
print("                                                                                      ")
#print(paramH4mod.orbitalEnergies())

In [ ]:
H5 = gqcpy.Molecule.HRingFromDistance(5, 1.889, 0)
structH5, energy_H5, paramH5 = GHF_calculation(H5, "sto-3g")

In [ ]:
print(paramH5.expansion().matrix())

## H6 ring vs RHF modified H6 ring

In [ ]:
# A H6 ring, 1 angstrom apart.
H6 = gqcpy.Molecule.HRingFromDistance(6, 1.889, 0)

In [ ]:
structH6, energy_H6, paramH6 = GHF_calculation(H6, "sto-3g")

In [ ]:
structH6mod, energy_H6mod, paramH6mod = GHF_calculation(H6, "sto-3g", "RHF")

### Compare coefficient matrices

In [ ]:
print(paramH6.expansion().matrix())
print("                                                                                      ")
print("======================================================================================")
print("                                                                                      ")
#print(paramH6mod.expansion().matrix())

### Compare orbital energies

In [ ]:
print(paramH6.orbitalEnergies())
print("                                                                                      ")
print("======================================================================================")
print("                                                                                      ")
print(paramH6mod.orbitalEnergies())